## This notebook-guide is illustrating the basic use-cases for the ChromaDB 

poetry install

In [5]:
# let's import all the things we need first
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders.pdf import PyPDFLoader

In [6]:
# load the document and split it into chunks

loader = PyPDFLoader("pdfs/Sports-And-Exercise-Nutrition.pdf")
documents = loader.load()

In [3]:
# split it into chunks
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=100,
    # length_function=len
)

texts = text_splitter.split_documents(documents)

Created a chunk of size 1307, which is longer than the specified 1000
Created a chunk of size 1346, which is longer than the specified 1000
Created a chunk of size 1419, which is longer than the specified 1000
Created a chunk of size 1102, which is longer than the specified 1000
Created a chunk of size 1230, which is longer than the specified 1000
Created a chunk of size 1248, which is longer than the specified 1000
Created a chunk of size 1496, which is longer than the specified 1000
Created a chunk of size 1050, which is longer than the specified 1000
Created a chunk of size 2026, which is longer than the specified 1000
Created a chunk of size 1118, which is longer than the specified 1000
Created a chunk of size 1029, which is longer than the specified 1000
Created a chunk of size 1571, which is longer than the specified 1000
Created a chunk of size 1631, which is longer than the specified 1000
Created a chunk of size 2110, which is longer than the specified 1000
Created a chunk of s

In [ ]:
texts[0].page_content

In [ ]:
# the final chunk is on the page 398 and of index 2153
texts[2153].metadata["page"]

In [ ]:
# accurately executed
texts[2153].page_content

In [ ]:
from langchain.embeddings.cohere import CohereEmbeddings
COHERE_API_KEY = "A1GXIK13oJjJ1ZgxeMHFhz38El38RRKMAxmNbL0l"
cohere_embeddings = CohereEmbeddings(cohere_api_key="A1GXIK13oJjJ1ZgxeMHFhz38El38RRKMAxmNbL0l") # type: ignore

In [4]:
db_using_cohere_embeddings = Chroma.from_documents(documents=texts, embedding=cohere_embeddings)

NameError: name 'cohere_embeddings' is not defined

In [ ]:
# query it
query = "What is the best time to eat before a workout?"

In [ ]:
docs =  db_using_cohere_embeddings.similarity_search(query)

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import Cohere

def load_chain(chain_type: str = "map_reduce"):
    """
    this function is loading the chain and sets it up for the agent to use
    :param chain_type: the type of chain to use
    :return: the LLM chain object
    """
    chain = load_qa_chain(
        Cohere(cohere_api_key=COHERE_API_KEY, verbose=True),  
        chain_type=chain_type,
        # Setting verbose to True will print out some internal states of the Chain object while it is being ran.
        verbose=False,
    )
    return chain

In [ ]:
chain = load_chain()
response = chain.run(input_documents=docs, question=query)

In [ ]:
print("RESPONSE FROM COHERE EMBEDDINGS: ", response)

In [ ]:
import chromadb
from chromadb.config import Settings

client = chromadb.Client(Settings(
    # chroma_db_impl=duckdb+parquet, 
    persist_directory=".chroma/"
))

client.reset()

In [ ]:
import chromadb
chroma_client = chromadb.HttpClient(host='localhost', port=8000)

In [ ]:
my_collection = client.create_collection("book_embeds_2")

In [ ]:
print(docs[0].page_content)

In [ ]:
ls = []
for doc in docs:
    ls.append(doc.page_content)


In [ ]:
my_embeddings = cohere_embeddings.embed_documents(ls)

In [ ]:
my_collection.add(
    documents=docs,
    embeddings=my_embeddings,
    ids=str([i for i in range(len(my_embeddings))])
)